# 실험 계획법 (Document of Experiment)

여러 변수들이 있고, 각 변수들마다 여러 수준이 있을 때, 어떤 변수의 어떤 레벨일 때 효과가 가장 최적인지를 판단하기 위한 방법론.


여러가지 방법이 있으며 대표적인 것으로는

1. Factorial Design
2. Fractional Factorial Design
3. Nested Design
등이 있다.

이때 Factorial Design이란...

RAG 파이프라인에서 변수를 
1. chunking size
2. retrieving document's top_k 
3. utilized embedding model

이라고 하고 

1번 factor의 경우 128, 256, 512, 1024로 4개의 treatment  
2번 factor의 경우 3, 5, 10, 20으로 4개의 treatment
3번 factor의 경우 text-embedding-2-ada, text-embedding-3-small, text-embedding-3-large 이렇게 세가지가 있다면   

4 * 4 * 3 = 48로 총 48번의 실험을 진행한 후, 그 결과를 ANOVA 분석을 통해서 분석하는 방법론이다.

장점은 factor들과 treatment들간의 상호작용을 아주 명확하게 볼 수 있다는 장점이 있지만 가장 큰 약점은 시간과 비용이 많이 든다는 점이다.

이를 보완하기 위해 이용되는 방법론으론 Fractional Factorial Design이란 것이 존재하는데, 이 방법은 모든 조합에 대한 실험을 진행하지 않고, contrast라는 개념을 이용해서 일부만을 실험해도 전체 factor들의 상호작용에 대한 어느정도의 분석을 진행할 수 있게끔 실험을 설계하여 진행하는 방법이다.  

자세하게 설명하려면 다소 수학적이고 과하게 이론적인 것 같아서,,, 1 2번에 대한 설명은 이쯤하고  

사실 우리가 주로 활용할 방법은 3번이다. 

## Nested Design

그럼 Nested Design은 뭐고, 어떤 경우에 주로 쓰이고 이걸 우리는 어떤식으로 쓸건데?

1. Nested Design이란 실험 단위들이 계층적으로 존재하는 경우에 주로 쓰이는 방법론이다. 
가령, 우리가 진행하고자 하는 RAG pipeline 최적화와 관련된 실험에서 위의 세가지 factor (chunk size, top_k, model)이 있다고 한다면, 세 요소는 실험 과정에서 다른 과정에 쓰인다. 가령, chunk_size는 처음에 문서를 쪼개서 Node를 만들 때 이용되고, top_k는 query가 입력됐을 때 검색하는 용도로 이용되며, model의 경우는 nodes가 쪼개졌을 때, 그것을 embedding해서 벡터 디비를 만들 때 이용된다. 

이러한 경우에는 사실 특정 RAG pipeline을 만들 때, 어떤 검색을 수행하고 어떤 querying을 하느냐에 따라서 최적인 chunk_size, top_k, model 등이 별개로 존재할 수도 있다. 그럼에도 불구하고 전체 factorial design을 하는 것은 과하게 비용소모적이며 비효율적일 수도 있다. 

이럴 때 nested design을 주로 이용하는데, 우리는 사실상 nested design만 쓰긴 할 것이다. 

2. 그럼 이건 어떻게 하는건데?
예를 들어서 설명을 하자면, 내가 교과서 서칭 최적화를 위한 연구에서 했던 방식은 다음과 같다. 

변수로는 [Retrieving방식, top_k, pipeline의사결정 방식]이 있었다. 
각각에 대해 좀 더 자세히 설명을 해보자면 
Retrieving 방식으론
1. Cosine Sim 이용
2. Dot Product 이용
3. BM25 이용
4. hybrid (BM25 + Cosine Sim) 이용 
의 4가지를 설정했고, 

top_k의 경우 1, 3, 5, 10, 20으로 5가지를 설정했으며   

의사결정 방식은 최종적인 의사결정을 할 때 
1. Majority Voting을 쓰는지
  a. 그리고 이 경우엔 threshold를 무엇으로 설정할 것인지.
2. Similarity Sum을 쓰는지
로 두가지를 이용했다.

이때 우리는 factorial design을 하게될 경우 불필요한 경우에 대한 실험을 너무 많이 진행해야하며 분산 분석 과정도 번거롭다는 판단하에 Nested Design을 실행했고, 보다 효율적인 실험 진행을 위해 다음과 같은 제한 조건을 걸어서 계층적인 실험을 진행했다. 

1. retrieving 방식과 의사결정 방식을 cosine-sim과 similarity-sum으로 고정해두고 실험을 우선 진행할 때는 모든 top_k에 대해서 실험을 진행한다.
2. 이때 도출된 결론 (어떤 top_k가 효율이 좋은가를 반영하여)을 기반으로 이후 실험에서는 불필요한 top_k에 대한 실험을 진행하지 않는다. 
3. Majority voting 방식을 이용하는 경우에는 말그대로 majority voting이기 때문에 top_k가 어느정도 커야 의미가 있기 때문에 top_k가 10, 20인 경우에 대해서만 실험을 진행한다.
4. Threshold를 정함에 있어서도 최초 실험에는 top_20, cosine-sim으로 다른 팩터들을 고정해둔 상태에서 threshold가 0.6일 때 가장 좋은 성능이 나와서 그것으로 고정해두고 나머지 실험 진행. (교수님한테도 지적받은 부분이지만,,, 사실 비약이 있긴 했음. 변수를 masking할 때는 적어도 두번은 실험을 진행하는걸 권한다고 합니다.)

나는 이렇게 실험을 설계 / 진행해서 순수 factorial design을 썼다면 총 80번 실험을 진행해야 했을텐데 nesting을 통해서 23번의 실험으로 결론을 도출했다.  


정리하자면 본인이 현재 해야하는 task가 무엇인지를 정하고, pipeline을 최적화하려면 어떤 변수를 설정해야하고 각 변수의 treatment를 어떻게 설정해야할지 의사결정하고, 그 의사결정 결과를 기반으로 실험을 설계하여 진행하는 과정을 **실험계획**이라고 이해하면 될 것 같습니다 ~

#### 그래서 우리가 진행할 실습은 뭘까요?

# SQuAD를 활용한 RAG pipeline optimization (우와~~ 짝짝)

데이터셋 소개   
-> Stanford 대학에서 제공하는 QA를 위한 데이터셋입니다. 
이 데이터는 title, context, answer, question, id로 구성되는 친구입니다. 
데이터를 전처리하는 과정까지는 제가 아래에 작성을 해뒀으니까 그거대로 해주시면 될 것 같고,  
전처리 된 데이터를 활용해서 좋은 RAG pipeline을 구축해주시면 됩니다! 

팩터도 자유롭게 설정하시면 되고, nesting도 자유롭게 해주시면 돼요. 어떻게든 최고의 성능을 만들어오면 됩니다.

이때 성능을 평가하는 기준은 아무래도 generator가 정답을 내뱉었냐가 돼야겠죠. 그러기 위해서 저희가 취할 방법은 다음과 같아요. 

ChatGPT-3.5-turbo 모델에게 generated된 응답과 answer를 같이 넘겨줘서 올바른 답을 내놨는지를 True, False로 받는 것입니다. 그렇게 해서 True의 비중이 가장 높은 pipeline을 구축해주시면 됩니다.

또 하나의 평가 요소는 강건성입니다. 모델이 얼마나 자신감을 가지고 판단을 했는가를 보기 위한 것이고 이를 측정하는 방법은 OpenAI logprob 기능을 이용해서 true에 대한 확률치를 평균내서 저는 이전 실험에 이용을 했었습니다.

그냥 제가 이용한 척도 그대로 이용하는걸로 하셔도 좋고, 아니면 다른 평가 기준을 만들고 싶다! 하시면 얼마든지 환영입니다. 

1. 실험의 조건은 최소한 세개 이상의 factor를 설정해주신다는 점 
2. nesting을 한다면 그 논리 과정이 명확히 드러나야 한다는 점   
3. 실험을 거치며 최적의 pipeline이 뭐였는지에 관한 명확한 결론이 나와야 한다는 점
4. 최소 8번의 실험 진행하기

이렇게 세가지로 생각을 해주시면 되겠습니다.

아 그리고 조건이 하나 더 있는데, 
1. ChatGPT-3.5-turbo, text-embedding-3-small 이렇게 두가지 모델만을 이용해주세요! 아무래도 실제로 이용할 파이프라인도 아니고 한데 굳이 좋은 모델 쓰면 돈 아까우니까,,, 일단 저 모델들로 해주세요!


변수가 될 요소들에 대해 힌트를 드리자면

1. top_k
2. retrieving 방식 (auto-merging, sentence-spliting, tokentextspliting, bm25 등등)
3. post-processing 방식
4. chunk_size 등등

In [ ]:
from llama_index.llms.openai import OpenAI as OpenAI_llama
from llama_index.core import Settings 
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI_llama(model='gpt-3.5-turbo')
Settings.embed_model = OpenAIEmbedding(model='text-embedding-3-small')

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
from datasets import load_dataset
from transformers import TrainingArguments, Trainer


dataset = load_dataset('squad')
train_data = dataset['train']
validation_data = dataset['validation']

In [ ]:
validation_data = validation_data.select(range(300))

In [ ]:
validation_data[0]

In [ ]:
validation_data[0]

In [ ]:
validation_data[1]

In [ ]:
len(validation_data)

In [ ]:
validation_data[2]

In [ ]:
import pandas as pd


data = {
    'context': [],
    'question': [],
    'answer': []
}


for item in validation_data:
    context = item['context']
    question = item['question']
    for answer in item['answers']['text']:
        data['context'].append(context)
        data['question'].append(question)
        data['answer'].append(answer)


df = pd.DataFrame(data)


print(df.head())

In [ ]:
df.head()

In [ ]:
import pandas as pd
from datasets import load_dataset

data = {
    'context': [],
    'question': [],
    'answer': []
}

for item in validation_data:
    context = item['context']
    question = item['question']
    answer = item['answers']['text'][0]
    data['context'].append(context)
    data['question'].append(question)
    data['answer'].append(answer)

df = pd.DataFrame(data)

output_path = "./squad_data.csv" 
df.to_csv(output_path, index=False)

print(f"DataFrame has been saved to {output_path}")



In [ ]:
df.head()

In [ ]:
contexts = []
previous_context = None

for item in validation_data:
    context = item['context']
    if context != previous_context:
        contexts.append(context)
        previous_context = context

output_path = "./squad_contexts.txt"  

with open(output_path, 'w', encoding='utf-8') as f:
    for context in contexts:
        f.write(context + '\n')

print(f"Contexts have been saved to {output_path}")

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter

documents = SimpleDirectoryReader(r"C:\Users\RSM\OneDrive\바탕 화면\API 스터디\train-v2.0").load_data()

text_splitter = TokenTextSplitter(
  chunk_size=512,
)

nodes = text_splitter(documents)

In [ ]:
for node in nodes:
  print(node)

In [ ]:
for node in nodes:
  print(node)

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes)

In [ ]:
index.storage_context.persist('./index_of_chunks_512')

In [ ]:
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(persist_dir='./index_of_chunks_512')
index = load_index_from_storage(storage_context)

In [ ]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response_synthesizers import ResponseMode

retriever = VectorIndexRetriever(
  index=index,
  similarity_top_k=3
)

response_synthesizer = get_response_synthesizer(
  response_mode=ResponseMode.COMPACT
)

query_engine = RetrieverQueryEngine(
  retriever=retriever,
  response_synthesizer=response_synthesizer
)

In [ ]:
from openai import OpenAI
import numpy as np 

client = OpenAI()

accuracy_list = []
robustness_list = []

for index, row in df.iterrows():
    question = row['question']
    answer = row['answer']
    
    predicted_answer = query_engine.query(question)
    
    is_answer = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role': 'system', 'content': 'You will be given a question, answer for that question and a predicted answer made by a model. You have to decide whether the predicted answer is correct or not. If it is right, your answer should be "True", otherwise "False". Remember that your answer must only be "True" or "False".'},
            {'role': 'assistant', 'content': f"Question : {question} \n Answer : {answer} \n Predicted Answer : {predicted_answer}"}
        ],
        temperature=0,
        logprobs=True,
        top_logprobs=2
    )
    
    top_logprobs = is_answer.choices[0].logprobs.content[0].top_logprobs
    
    is_correct = top_logprobs[0].token
    
    if is_correct == 'True':
        robustness = np.exp(top_logprobs[0].logprob)
        accuracy = 'True'
    else:
        robustness = np.exp(top_logprobs[1].logprob)
        accuracy = 'False'
    
    accuracy_list.append(accuracy)
    robustness_list.append(robustness)

df['accuracy'] = accuracy_list
df['robustness'] = robustness_list

df.to_csv('./chunk_size_512.csv', index=False)


In [ ]:
import pandas as pd

data = {
    'accuracy': ['True', 'False', 'True', 'True', 'False', 'True'],
    'robustness': [0.8, 0.5, 0.9, 0.7, 0.6, 0.85]
}

df = pd.DataFrame(data)

df['accuracy'] = df['accuracy'].map({'True': True, 'False': False})

true_ratio = df['accuracy'].mean()

mean_robustness = df['robustness'].mean()

print(f"True Ratio: {true_ratio}")
print(f"Mean Robustness: {mean_robustness}")
